In [35]:
import pandas as pd
import datetime as dt

dataset = pd.read_csv('./data/BTC-Daily.csv')

# convert column date of dataset to datetime
dataset['date'] = pd.to_datetime(dataset['date'])

# filter dataset to get data from 2015-01-01 to 2016-12-31
dataset = dataset[(dataset['date'] >= dt.datetime(2015, 1, 1)) & (dataset['date'] <= dt.datetime(2020, 12, 31))]

# describe dataset
dataset.describe()

,unix,open,high,low,close,Volume BTC,Volume USD
count,2.192000e+03,2192.000000,2192.000000,2192.000000,2192.000000,2.192000e+03,2.192000e+03
mean,1.514722e+09,5123.166638,5272.377637,4960.782660,5136.057441,1.604049e+07,3.488562e+07
std,5.468431e+07,4798.694398,4955.226070,4633.449376,4824.461973,5.143656e+07,5.614624e+07
min,1.420070e+09,162.000000,212.840000,152.400000,162.000000,0.000000e+00,0.000000e+00
25%,1.467396e+09,579.737500,588.037500,572.852500,581.760000,7.283284e+03,8.078573e+03
50%,1.514722e+09,4097.235000,4208.365000,3958.025000,4099.775000,9.328728e+05,2.759654e+04
75%,1.562047e+09,8629.675000,8835.342500,8314.757500,8629.715000,4.648965e+06,5.225920e+07
max,1.609373e+09,28893.210000,29300.000000,27930.750000,28992.790000,7.723295e+08,5.162108e+08


In [36]:
import numpy as np

# preserve only open, high, low, close, volume usd columns
dataset = dataset[['open', 'high', 'low', 'close', 'Volume USD']]
dataset.describe()

,open,high,low,close,Volume USD
count,2192.000000,2192.000000,2192.000000,2192.000000,2.192000e+03
mean,5123.166638,5272.377637,4960.782660,5136.057441,3.488562e+07
std,4798.694398,4955.226070,4633.449376,4824.461973,5.614624e+07
min,162.000000,212.840000,152.400000,162.000000,0.000000e+00
25%,579.737500,588.037500,572.852500,581.760000,8.078573e+03
50%,4097.235000,4208.365000,3958.025000,4099.775000,2.759654e+04
75%,8629.675000,8835.342500,8314.757500,8629.715000,5.225920e+07
max,28893.210000,29300.000000,27930.750000,28992.790000,5.162108e+08


In [37]:
# extract close column
close = dataset['close'].copy()

# drop close column from dataset
dataset.drop('close', axis=1, inplace=True)

In [38]:
dataset.describe()

,open,high,low,Volume USD
count,2192.000000,2192.000000,2192.000000,2.192000e+03
mean,5123.166638,5272.377637,4960.782660,3.488562e+07
std,4798.694398,4955.226070,4633.449376,5.614624e+07
min,162.000000,212.840000,152.400000,0.000000e+00
25%,579.737500,588.037500,572.852500,8.078573e+03
50%,4097.235000,4208.365000,3958.025000,2.759654e+04
75%,8629.675000,8835.342500,8314.757500,5.225920e+07
max,28893.210000,29300.000000,27930.750000,5.162108e+08


In [39]:
close.describe()

count     2192.000000
mean      5136.057441
std       4824.461973
min        162.000000
25%        581.760000
50%       4099.775000
75%       8629.715000
max      28992.790000
Name: close, dtype: float64

In [40]:
# convert labels to boolean
close = close > close.shift(-1)
close = np.array([int(c) for c in close])

In [41]:
# enhance the dataset with the data from the previous n days
n = 15

dataset_enhanced = np.array(
    [ 
        np.column_stack([dataset.to_numpy()[j,:] for j in range(i-n, i)]).flatten()
        for i in range(n, len(dataset))
    ]
)
labels = close[n:]

# print shape of dataset_enhanced and labels
print(dataset_enhanced.shape)
print(labels.shape)

(2177, 60)
(2177,)


In [42]:
# split dataset into training and test set and classify them with a linear SVM
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset_enhanced, labels, test_size=0.2, random_state=42)



In [43]:
# try a linear SVC classifier

from sklearn.svm import LinearSVC

clf = LinearSVC(random_state=0, tol=1e-5, max_iter=1e4)
clf.fit(X_train, y_train)

# print accuracy of the classifier
print(clf.score(X_test, y_test))

0.48394495412844035


/Users/giacomocartechini/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [44]:
# try a hoeffding tree classifier

from skmultiflow.trees import HoeffdingTree

clf = HoeffdingTree()
clf.fit(X_train, y_train)

# print accuracy of the classifier
print(clf.score(X_test, y_test))

/Users/giacomocartechini/anaconda3/lib/python3.9/site-packages/skmultiflow/trees/hoeffding_tree.py:34: FutureWarning: 'HoeffdingTree' has been renamed to 'HoeffdingTreeClassifier' in v0.5.0.
The old name will be removed in v0.7.0
  warnings.warn("'HoeffdingTree' has been renamed to 'HoeffdingTreeClassifier' in v0.5.0.\n"


0.518348623853211


In [45]:
# naive model that predicts the last observation from the training data

y_pred = [y_test[i-1] for i in range(0, len(y_test))]

acc = sum([1 if y_pred[i] == y_test[i] else 0 for i in range(0, len(y_test))]) / len(y_test)

print('Accuracy:', acc)

Accuracy: 0.5045871559633027


In [59]:
# try a random forest classifier

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1000, max_depth=2, random_state=0)
clf.fit(X_train, y_train)

# print accuracy of the classifier
print(clf.score(X_test, y_test))

0.5206422018348624


In [65]:
# build a streaming random patches model

from skmultiflow.meta import StreamingRandomPatchesClassifier

clf = StreamingRandomPatchesClassifier()
clf.fit(X_train, y_train)

# print accuracy of the classifier
print(clf.score(X_test, y_test))

0.5160550458715596
